# Prueba para científico de datos

## Instrucciones

Por favor, siga al siguiente enlace y siga las instrucciones. Una vez haya terminado, continue con el notebook.

FALTA EL ENLACE ACA

Puede utilizar cualquier librería que desee a menos que se especifique lo contrario. 

## Parte 1 - Cargar datos y estadísticas básicas

En la carpeta ```Data``` encontrará un archivo llamado ```diamonds.csv```. Este archivo contiene información de 53940 diamantes. Dentro de la información disponible, está el precio, el color, el peso, etc. Puede consultar las características completas del dataset en [este enlace](https://www.kaggle.com/shivam2503/diamonds).

1. Cargue el archivo en un dataframe de pandas 
2. Use los metodos que conozca para describir las propiedades básicas de los datos. Le será útil para las siguientes secciones.

In [12]:
# Respuesta a la parte 1



## Parte 2 - Aprendizaje no supervisado

Usted desea encontrar estructura en los datos que le han sido dados. 
1. A partir del dataframe que cargó en el primer punto, use algún algoritmo de aprendizaje no supervisado para encontrar clusters de diamantes con propiedades similares. 
2. En una celda de markdown, describa una métrica/método que se pueda utilizar para evaluar qué tan bien sus clusters describen los datos. 
3. Varie $k$ (a cantidad de clusters) de 1 a 10 y grafique su métrica en función de $k$
4. Qué $k$ describe mejor sus datos? 

In [4]:
# Respuesta a la parte 2 




## Parte 3 - Regresión

